In [463]:
import pandas as pd
import csv
from functools import reduce
from multiprocessing import Pool

In [464]:
!wget -O housingSalseSample.csv https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv

--2022-02-14 16:02:40--  https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CUSP2022ADS/Data/main/housingSalseSample.csv [following]
--2022-02-14 16:02:40--  https://raw.githubusercontent.com/CUSP2022ADS/Data/main/housingSalseSample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6167298 (5.9M) [text/plain]
Saving to: ‘housingSalseSample.csv’

housingSalseSample. 100%[===================>]   5.88M  --.-KB/s    in 0.06s   

2022-02-14 16:02:41 (94.8 MB/s) - ‘housingSalseSample.csv’ saved [6167298/6167298]



In [546]:
house = pd.read_csv('housingSalseSample.csv')
house.head()

,BOROUGH,BLOCK,LOT,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,1170,1010,17 CONDOPS,"200 WEST 79TH STREET, 7P","10,024.00",0.00,"1,975.00",551000,2011-07-28
1,4,1275,45,10 COOPS - ELEVATOR APARTMENTS,"35-35 75TH STREET, 304","11,372.00",0.00,"1,941.00",338000,2011-05-20
2,3,5746,58,03 THREE FAMILY DWELLINGS,1249 65TH STREET,"11,219.00","3,861.00","1,910.00",1250000,2014-12-04
3,4,2086,50,10 COOPS - ELEVATOR APARTMENTS,"97-07 63 ROAD, 8F","11,374.00",0.00,"1,955.00",253000,2016-03-09
4,4,5028,7,10 COOPS - ELEVATOR APARTMENTS,"38-15 149TH STREET, 1F","11,354.00",0.00,"1,955.00",0,2016-12-30


In [547]:
def returnInfo(record): #extract necessary fields from a data record (raw)
    return (record['ZIP CODE'],record['GROSS SQUARE FEET'],record['SALE PRICE'])

def filterRec(record):
    size = record[1]
    price = record[2]
    if size > '500' and price > '100000':
        return True
    else:
        return False

def groupZip(result,record):
    zip = record[0]
    price = record[2]
    result[(zip)] = result.get(zip,0) + float(price)
    return result

def avgPrice(record):
    zip = record[0]
    price = [record[1]]
    avg = sum(price) / len(price)
    return (zip, avg)

In [549]:
with open('housingSalseSample.csv', 'r') as data:
    reader = csv.DictReader(data)
    output1 = list(map(returnInfo,reader))
output1[:5]

[('10024.0', '0.0', '551000'),
 ('11372.0', '0.0', '338000'),
 ('11219.0', '3861.0', '1250000'),
 ('11374.0', '0.0', '253000'),
 ('11354.0', '0.0', '0')]

In [550]:
output2 = list(filter(filterRec, output1))
output2[:5]

[('11214.0', '5054.0', '2000000'),
 ('10024.0', '5301.0', '4550000'),
 ('11215.0', '7660.0', '10775000'),
 ('11236.0', '896.0', '425000'),
 ('11217.0', '69998.0', '52200000')]

In [551]:
#output3 = list(reduce(groupZip,output2,{}).items())
#output3[:5]

In [552]:
#output4 = list(map(avgPrice,output3))
#output4[:5]

In [553]:
#with open('housingSalseSample.csv', 'r') as data: #with a given dataset
#    reader = csv.DictReader(data) #line-by-line data reader
#    output = list(map(lambda x: (x[0],x[1]), #convert to tuples
#                      map(avgPrice,
#                          list(reduce(groupZip,
#                                      filter(filterRec,map(returnInfo,reader) #extract necessary fields from the records
#                                             ),
#                                      ),
#                               {}).items()))) #initial value for the dictionary - {}

In [554]:
df = pd.DataFrame.from_records(output2, columns = ['zip','size','price'])
df = df.drop(columns=['size'])
df = df.astype({'price': float})
gdf = df.groupby('zip')[['price']].mean().reset_index()
ndf = gdf.rename(columns={'price':'avgprice'})
fdf = ndf.round({'avgprice': 2})
fdf.head()
records = fdf.to_records(index=False)
output3 = list(records)
output3[:5]

[('10001.0', 49613816.),
 ('10002.0', 3989431.81),
 ('10003.0', 5160787.79),
 ('10004.0', 2023333.5),
 ('10005.0', 859285.71)]

In [560]:
output = list(map(lambda x: (x[0],x[1]), output3))

In [561]:
def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)

grading(output)

In [562]:
from google.colab import files
files.download('output1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [538]:
!wget -O ADSSession2Task2Data.csv https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv

--2022-02-14 16:51:52--  https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279369 (273K) [text/plain]
Saving to: ‘ADSSession2Task2Data.csv’

ADSSession2Task2Dat 100%[===================>] 272.82K  --.-KB/s    in 0.02s   

2022-02-14 16:51:52 (15.3 MB/s) - ‘ADSSession2Task2Data.csv’ saved [279369/279369]



In [ ]:
with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    result = []
    # read 100000 rows in each chunk
    chunksize = 100000
    pool = Pool(8)
    for i, line in enumerate(reader):
        
        if (i % chunksize == 0 and i > 0):
            # MapReduce function for rows in one chunk
            output = list(pool.map(avgPrice,
                         reduce(groupZip,
                             pool.map(returnInfo,chunk)
                            ),
                         {}).items())
            result += output
            chunk = []
            
        
        # add lines to chunk, and when the index
        # is 100000, 200000, etc, stop adding new lines
        # and start MapReduce functions
        chunk.append(line)
        
    # MapReduce function for the last chunk
    output = list(pool.map(avgPrice,
                         reduce(groupZip,
                             pool.map(returnInfo,chunk)
                            ),
                         {}).items())
    result += output
    pool.close()

In [ ]:
def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(output)